In [1]:
import re
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [2]:
db_ip = '192.168.1.21'
db_port = 27017
db_kline_name = 'ricequant'
db_hunter_name = 'hunter'
col_stock_sector_name = 'stock_sector'
col_market_sector_name = 'market_sector'

client = MongoClient(db_ip, db_port)
db_kline = client[db_kline_name]
code_list = db_kline.collection_names(include_system_collections=False)

col_stock_sector = client[db_hunter_name][col_stock_sector_name]
col_market_sector = client[db_hunter_name][col_market_sector_name]

In [3]:
base_url = 'http://basic.10jqka.com.cn/'
company_post_url = '/company.html'
concept_post_url = '/concept.html'

In [4]:
def get_stock_industry(code):
    res = requests.get(base_url + code + company_post_url)
    soup = BeautifulSoup(res.text, 'lxml')
    table = soup.find('table', attrs={'class':'m_table'})
    industry = table.find_all('tr')[1].find_all('td')[1].findAll(text=True)[1]
    pattern = re.compile('.*\w.*')
    if pattern.search(industry):
        industry = industry.split(' — ')
        industry = list(set(industry))
    else:
        print('Can not find industry for {}, raw industry: {}'.format(code, industry))
        industry = []
    return industry

def get_stock_concept(code):
    res = requests.get(base_url + code + concept_post_url)
    soup = BeautifulSoup(res.text, 'lxml')
    tables = soup.find_all('table', attrs={'class':'gnContent'})
    normal_concept = []
    normal_tables = soup.find_all('div', attrs={'id':'concept'})[0].find_all('table', attrs={'class':'gnContent'})
    if len(normal_tables) > 0:
        for item in normal_tables[0].find_all('td', attrs={'class':'gnName'}):
            normal_concept.append(item.text.strip())
    other_concept = []
    other_div = soup.find_all('div', attrs={'id':'other'})
    if len(other_div) > 0:
        other_tables = other_div[0].find_all('table', attrs={'class':'gnContent'})
        if len(other_tables) > 0:
            for item in other_tables[0].find_all('td', attrs={'class':'gnStockList'}):
                other_concept.append(item.text.strip())
    return normal_concept, other_concept

In [5]:
stock_category = []
for idx, code in enumerate(code_list):
    simple_code = code.split('.')[0]
    ths_industry = get_stock_industry(simple_code)
    ths_normal_concept, ths_other_concept = get_stock_concept(simple_code)
    stock_info = {'code': code,
                  'ths_industry': ths_industry,
                  'ths_normal_concept': ths_normal_concept,
                  'ths_other_concept': ths_other_concept}
    stock_category.append(stock_info)
    print('{} {}: {} {} {}'.format(idx, code, ths_industry, ths_normal_concept, ths_other_concept))
print('Finished')

0 002555.XSHE: ['传媒', '信息服务'] ['手机游戏', '网络游戏', '参股新三板', '文化传媒', '虚拟现实'] []
1 600208.XSHG: ['房地产开发', '房地产'] ['参股券商', '互联网金融', '金改', '期货概念', '参股民营银行', 'MSCI概念'] ['涉矿', '保障房']
2 000785.XSHE: ['零售', '商业贸易'] ['O2O概念'] ['地方国资改革', '武汉自贸区']
3 600216.XSHG: ['化学制药', '医药生物'] ['医疗改革', '禽流感', '证金持股'] ['诺贝尔', '青蒿素', '疟疾', '抗癌', '医保']
4 000828.XSHE: ['交通运输', '公路铁路运输'] ['金改', '参股券商', '参股新三板'] ['参股新股', '春运', '融资租赁', '地方国资改革', '并购重组']
5 002110.XSHE: ['钢铁', '黑色金属'] ['核电', '电力改革'] ['并购重组', '地方国资改革', '海峡西岸']
6 002385.XSHE: ['农产品加工', '农林牧渔'] ['互联网金融', '农村电商', '移动支付', '猪肉', '互联网+', 'MSCI概念'] ['征信', '饲料', '生物育种']
7 300149.XSHE: ['农产品加工', '农林牧渔'] ['二胎概念', '股权转让'] ['并购重组', '生物农药']
8 603609.XSHG: ['农产品加工', '农林牧渔'] ['生态农业'] []
9 603501.XSHG: ['半导体及元件', '电子'] ['新股与次新股', '集成电路'] []
10 000783.XSHE: ['金融服务', '证券'] ['参股新三板', '举牌', 'MSCI概念'] []
11 600478.XSHG: ['新材料', '有色金属'] ['锂电池', '稀土永磁', '新能源汽车', '新能源', '新材料概念'] ['绿色消费', '涉矿']
12 000532.XSHE: ['金融服务', '保险及其他'] ['无人机', '举牌', '创投', '石墨烯', '参股新三板'] ['互联网平台', '地方国资改革',

116 600587.XSHG: ['医疗器械服务', '医药生物'] ['医疗器械', '互联网+', '证金持股'] ['互联网医疗', '地方国资改革', '医保']
117 002197.XSHE: ['计算机设备', '信息设备'] ['互联网金融', '金融IC', '二维码识别', '移动支付', '云计算', '网络安全', '互联网+', '微信小程序'] ['微信概念', '移动金融', '互联网保险', '换芯', '移动pos机']
118 601111.XSHG: ['交通运输', '机场航运'] ['央企国资改革', '中字头股票', 'MSCI概念'] ['人民币升值', '油价下调']
119 002405.XSHE: ['计算机应用', '信息服务'] ['阿里巴巴概念', '车联网', '马云概念', '电子信息', '智慧城市', '卫星导航', '特斯拉', '无人驾驶', '汽车电子', '互联网+', '证金持股', '人工智能', '网约车'] ['腾讯概念', '智能汽车', '国产替代', '智能交通', '地理信息']
120 601616.XSHG: ['电气设备', '机械设备'] ['智能电网'] []
121 002239.XSHE: ['汽车零部件', '交运设备'] ['新能源汽车'] ['汽车制造概念', '通州湾新区', '大上海', '涉矿']
122 300348.XSHE: ['计算机应用', '信息服务'] ['互联网金融', '智慧城市'] ['腾讯概念', '互联网银行']
123 000953.XSHE: ['化工', '化学制品'] ['股权转让', 'ST板块'] []
124 300184.XSHE: ['其他电子', '电子'] ['能源互联网', '物联网', '车联网', '电子商务', '智能穿戴'] []
125 603100.XSHG: ['电气设备', '机械设备'] ['高端装备', '核电'] ['地方国资改革']
126 300266.XSHE: ['公用事业', '环保工程'] ['美丽中国', '污水处理', 'PPP概念', '节能环保'] ['并购重组']
127 600460.XSHG: ['光学光电子', '电子'] ['节能照明', '集成电路'

228 300428.XSHE: ['新材料', '有色金属'] ['新材料概念', '证金持股', '雄安新区'] ['汽车制造概念']
229 002144.XSHE: ['纺织服装', '纺织制造'] ['互联网金融', '医疗器械', '互联网+', '证金持股'] []
230 000965.XSHE: ['房地产开发', '房地产'] ['京津冀一体化', '天津自贸区', '大飞机', '参股券商', '雄安新区'] ['地方国资改革']
231 002008.XSHE: ['其他电子', '电子'] ['蓝宝石', '3D打印', '机器人概念', '节能照明', '苹果概念', '养老概念'] ['光纤', '引力波']
232 002783.XSHE: ['化工', '化学制品'] ['军工', '参股新三板', '军民融合'] ['地方国资改革']
233 002866.XSHE: ['电子制造', '电子'] ['新股与次新股'] []
234 300484.XSHE: ['电气设备', '机械设备'] ['新能源汽车', '工业4.0'] ['新能源物流车']
235 600103.XSHG: ['轻工制造', '造纸'] ['参股新三板'] ['地方国资改革', '人民币升值', '海峡西岸']
236 000923.XSHE: ['专用设备', '机械设备'] ['创投', '京津冀一体化', '雄安新区'] ['涉矿', '铜冶炼', '地方国资改革', '冬奥会']
237 600962.XSHG: ['农产品加工', '农林牧渔'] ['央企国资改革'] []
238 300065.XSHE: ['计算机设备', '信息设备'] ['海工装备', '军工', '电子信息'] ['并购重组', '海洋经济']
239 603778.XSHG: ['建筑材料', '建筑装饰'] ['节能环保', 'PPP概念'] []
240 603838.XSHG: ['家用轻工', '轻工制造'] [] []
241 000910.XSHE: ['家用轻工', '轻工制造'] [] ['北京环球影城']
242 002243.XSHE: ['包装印刷', '轻工制造'] ['深圳国资改革', '迪士尼'] ['富勒烯', '地方国资改革']
24

342 300102.XSHE: ['光学光电子', '电子'] ['节能照明', '卫星导航', '举牌', '股权转让'] []
343 600688.XSHG: ['基础化学', '化工'] ['央企国资改革', '两桶油改革', '油品改革', '证金持股', 'MSCI概念'] []
344 600517.XSHG: ['电气设备', '机械设备'] ['央企国资改革', '新能源', '特高压', '智能电网'] ['碳交易', '并购重组']
345 002168.XSHE: ['电气设备', '机械设备'] ['3D打印', '碳纤维', '智能电网', '新能源', '锂电池', '大数据', '新材料概念', '手机游戏', 'OLED', '体育产业', '证金持股', '股权转让'] ['并购重组', '能源纸', '智能纤维', '碳纳米管', '海水淡化', '柔性电子', '膜材料', '超级电容']
346 300412.XSHE: ['医疗器械服务', '医药生物'] ['工业4.0', '证金持股'] []
347 603768.XSHG: ['汽车零部件', '交运设备'] ['新股与次新股'] ['汽车制造概念']
348 603920.XSHG: ['半导体及元件', '电子'] ['新股与次新股', 'OLED'] []
349 002838.XSHE: ['化工', '化工合成材料'] ['新股与次新股', '新材料概念'] []
350 300500.XSHE: ['建筑材料', '建筑装饰'] ['建筑节能', '雄安新区'] []
351 000537.XSHE: ['房地产开发', '房地产'] ['央企国资改革', '土地流转', '高校'] ['并购重组']
352 300600.XSHE: ['电气设备', '机械设备'] ['军工', '新股与次新股', '军民融合'] []
353 000977.XSHE: ['计算机设备', '信息设备'] ['军工', '云计算', '国产软件', '网络游戏', '电子信息', '智慧城市', '电子发票', '人工智能'] ['地方国资改革', '国产替代', '去IOE', 'IPV6', '超级计算机']
354 002536.XSHE: ['汽车零部件',

452 002798.XSHE: ['家用轻工', '轻工制造'] ['参股新三板'] ['并购重组']
453 600297.XSHG: ['交运设备服务', '交运设备'] ['医疗改革', '证金持股', 'MSCI概念'] ['并购重组', '融资租赁', '汽车制造概念', '鼠疫', '眼科医疗', '抗癌', '抗辐射药']
454 600738.XSHG: ['零售', '商业贸易'] [] ['并购重组', '兰州自贸区']
455 601558.XSHG: ['电气设备', '机械设备'] ['风电', '新能源', 'ST板块', '高端装备'] ['海上风电']
456 601608.XSHG: ['专用设备', '机械设备'] ['工业4.0', '高端装备', '机器人概念', '新能源汽车', '节能环保', '央企国资改革'] ['外汇局持股', '节能减排']
457 002501.XSHE: ['有色金属', '有色冶炼加工'] ['高铁', '高端装备', '军工', '特斯拉'] ['轨道交通']
458 603038.XSHG: ['家用轻工', '轻工制造'] ['新股与次新股'] []
459 603380.XSHG: ['电子制造', '电子'] ['新股与次新股', '汽车电子'] []
460 603608.XSHG: ['服装家纺', '纺织服装'] ['电子商务', '移动互联网'] ['互联网广告', '她经济']
461 600399.XSHG: ['钢铁', '黑色金属'] ['军工', '特钢', '大飞机', '振兴东北'] ['股权冻结', '地方国资改革', '航天军工']
462 300208.XSHE: ['电气设备', '机械设备'] ['举牌', '小金属', '煤化工', '智能电网', '一带一路'] ['融资租赁', '厄尔尼诺概念', '轨道交通']
463 002654.XSHE: ['光学光电子', '电子'] ['节能照明', '互联网+', '参股新三板', '文化传媒'] ['互联网广告']
464 002100.XSHE: ['农产品加工', '农林牧渔'] ['禽流感', '新疆振兴', '物联网', '猪肉'] ['兽药', '生物疫苗']
465 300144.X

...

2949 600199.XSHG: ['食品饮料', '饮料制造'] ['白酒', '证金持股'] ['地方国资改革', '春节概念']
2950 600825.XSHG: ['传媒', '信息服务'] ['上海国资改革', '文化传媒', '网络游戏', '证金持股', '租售同权'] ['地方国资改革']
Can not find industry for 601299, raw industry: -
2951 601299.XSHG: [] [] []
2952 601718.XSHG: ['服装家纺', '纺织服装'] ['央企国资改革', '军工', '证金持股', '军民融合', 'MSCI概念'] ['阅兵', '航空航天']
2953 300319.XSHE: ['半导体及元件', '电子'] [] ['并购重组', '触摸屏']
2954 603099.XSHG: ['餐饮旅游', '景点及旅游'] ['央企国资改革'] ['冬奥会']
2955 600318.XSHG: ['金融服务', '保险及其他'] ['互联网金融', '移动支付', '新材料概念', 'P2P概念', '互联网+', '债转股'] ['融资租赁', '新型城镇化', '保障房']
2956 002841.XSHE: ['光学光电子', '电子'] ['新股与次新股', '智能穿戴', '在线教育', '人工智能'] []
2957 002053.XSHE: ['食品饮料', '食品加工制造'] ['风电', '天然气'] ['食盐', '调味品']
2958 002469.XSHE: ['建筑材料', '建筑装饰'] ['节能环保', '油品升级', '煤化工', '证金持股'] []
2959 600705.XSHG: ['金融服务', '保险及其他'] ['央企国资改革', '参股券商', '3D打印', '上海自贸区', '证金持股', '万达私有化', 'MSCI概念'] ['金控平台', '融资租赁']
2960 600797.XSHG: ['计算机应用', '信息服务'] ['国产软件', '阿里巴巴概念', '高校', '网络游戏', '电子信息', '云计算', '大数据', '互联网+', '人工智能', '蚂蚁金服概念'] ['互联网医疗', 'G2

3060 000030.XSHE: ['汽车零部件', '交运设备'] ['特斯拉', '汽车电子'] ['地方国资改革', '并购重组']
3061 601005.XSHG: ['钢铁', '黑色金属'] ['中韩自贸区', '特钢', 'ST板块', '证金持股'] ['重庆自贸区', '地方国资改革', '并购重组', '铁矿石']
3062 000931.XSHE: ['化学制药', '医药生物'] ['京津冀一体化', '创投', '参股新三板', '生物医药', '参股民营银行', '证金持股', '雄安新区'] ['医保', '众筹概念', '黄光裕概念', '抗辐射药']
3063 600377.XSHG: ['交通运输', '公路铁路运输'] [] ['春运', '地方国资改革']
3064 600759.XSHG: ['采掘', '石油矿业开采'] ['参股保险', '土地流转', '油品改革', '证金持股'] ['涉矿']
3065 002524.XSHE: ['公用事业', '燃气水务'] ['一带一路', '天然气', '参股保险'] ['钢结构', '新型城镇化']
3066 002733.XSHE: ['电气设备', '机械设备'] ['光伏概念', '能源互联网', '锂电池', '新能源汽车'] []
3067 600791.XSHG: ['房地产开发', '房地产'] ['京津冀一体化'] ['地方国资改革', '国资整合', '北京国资改革']
3068 603729.XSHG: ['传媒', '信息服务'] ['文化传媒'] []
3069 002770.XSHE: ['食品饮料', '食品加工制造'] ['乳业', '电子商务', '二胎概念'] []
3070 600634.XSHG: ['通信服务', '信息服务'] ['网络游戏', '文化传媒', '手机游戏', '股权转让'] ['航天军工']
3071 002285.XSHE: ['房地产开发', '房地产'] ['养老概念', '参股新三板', '租售同权', '债转股'] ['共享办公', '互联网平台', '养老地产', '老龄化']
3072 002394.XSHE: ['纺织服装', '纺织制造'] [] ['大上海']
3073 002035.XSH

3171 002505.XSHE: ['养殖业', '农林牧渔'] ['农业现代化', '乳业', '猪肉', '电子商务'] ['牛羊肉', '家庭农场']
3172 300324.XSHE: ['计算机应用', '信息服务'] ['网络安全', '手机游戏', '军工', '智慧城市', '大数据', '参股民营银行'] []
3173 000585.XSHE: ['电气设备', '机械设备'] ['智能电网', '振兴东北', '股权转让'] []
3174 000786.XSHE: ['建筑材料'] ['央企国资改革', '参股新三板', '建筑节能', '新材料概念', '雄安新区', '装配式建筑'] ['绿色消费', '并购重组', '地方国资改革', '保障房']
3175 300434.XSHE: ['专用设备', '机械设备'] ['地下管网', '生物医药'] []
3176 300455.XSHE: ['计算机应用', '信息服务'] ['高铁'] []
3177 300636.XSHE: ['化学制药', '医药生物'] ['新股与次新股'] []
3178 600203.XSHG: ['光学光电子', '电子'] ['超导', '物联网', '节能照明'] ['并购重组', '地方国资改革', '指纹技术']
3179 600455.XSHG: ['计算机应用', '信息服务'] ['举牌', '职业教育'] ['地方国资改革']
3180 600272.XSHG: ['服装家纺', '纺织服装'] ['上海国资改革', '生物医药'] ['地方国资改革', '老字号']
3181 600483.XSHG: ['公用事业', '电力'] ['核电'] ['能源纸', '地方国资改革', '海峡西岸', '并购重组']
3182 600584.XSHG: ['半导体及元件', '电子'] ['集成电路', '智能穿戴', '新材料概念', '移动支付', '苹果概念', '物联网', '芯片替代', '证金持股'] ['融资租赁', 'IGBT']
3183 600606.XSHG: ['房地产开发', '房地产'] ['上海自贸区', '足球概念', '互联网金融', 'PPP概念', '证金持股', '租售同权', '蚂蚁金服概念', 

3275 603136.XSHG: ['餐饮旅游', '景点及旅游'] ['新股与次新股'] []
3276 300567.XSHE: ['仪器仪表', '机械设备'] ['苹果概念', '新股与次新股', 'OLED'] []
3277 002652.XSHE: ['建筑材料'] ['新材料概念', '建筑节能'] []
3278 002077.XSHE: ['园区开发', '房地产'] ['节能环保', '光伏概念', '新能源', '军工', '锂电池', '集成电路'] ['引力波', '地方国资改革']
3279 300475.XSHE: ['家用电器', '白色家电'] ['家用电器'] []
3280 600642.XSHG: ['公用事业', '电力'] ['天然气', '新能源', '参股券商', '风电', '核电', '太阳能', '证金持股', 'MSCI概念'] ['地方国资改革', '西气东输', '油价下调']
3281 601988.XSHG: ['银行', '金融服务'] ['中字头股票', '证金持股', 'MSCI概念'] ['外汇局持股', 'SDR概念', '优先股概念']
3282 000886.XSHE: ['交通运输', '公路铁路运输'] [] []
3283 601137.XSHG: ['有色金属', '有色冶炼加工'] ['太阳能', '新能源', '新材料概念'] []
3284 000700.XSHE: ['汽车零部件', '交运设备'] ['民营医院', '无人驾驶'] []
3285 000650.XSHE: ['中药', '医药生物'] [] ['儿童用药', '医保', '眼科医疗']
3286 603258.XSHG: ['通信服务', '信息服务'] ['网络游戏', '新股与次新股', '手机游戏', '阿里巴巴概念', '虚拟现实'] ['腾讯概念']
3287 000612.XSHE: ['有色金属', '有色冶炼加工'] ['天然气', '稀缺资源', '举牌', '股权转让'] ['油气开采']
3288 600877.XSHG: ['非汽车交运', '交运设备'] ['央企国资改革', '中字头股票', 'ST板块', '军民融合'] ['军工改制', '航天军工', '国资整合']


3386 600317.XSHG: ['交通运输', '港口航运'] ['一带一路', '中韩自贸区', '振兴东北', '证金持股'] ['地方国资改革']
3387 000929.XSHE: ['食品饮料', '饮料制造'] ['举牌', '污水处理'] ['并购重组', '兰州自贸区', '啤酒']
3388 300479.XSHE: ['计算机应用', '信息服务'] ['移动支付', '网络安全', '人脸识别', '人工智能'] ['G20峰会', '互联网医疗']
3389 600711.XSHG: ['有色金属', '有色冶炼加工'] ['黄金', '锂电池', '小金属', '互联网金融', '供应链金融'] ['融资租赁', '涉矿', '并购重组']
3390 300077.XSHE: ['半导体及元件', '电子'] ['集成电路', '4G5G', '移动互联网', '金融IC', '移动支付', '物联网', '网络安全', '芯片替代'] ['交通一卡通', '移动金融', '国产替代', 'SNS概念', '换芯']
3391 002273.XSHE: ['光学光电子', '电子'] ['智能穿戴', '虚拟现实', '移动互联网', '苹果概念', '蓝宝石', '证金持股'] ['全息手机', '智能眼镜']
3392 002819.XSHE: ['仪器仪表', '机械设备'] ['新股与次新股', '央企国资改革'] ['中科院概念']
3393 600423.XSHG: ['化工', '化学制品'] ['煤化工', 'ST板块'] ['股权冻结', '地方国资改革']
3394 002234.XSHE: ['养殖业', '农林牧渔'] ['生态农业', '生物质能'] []
3395 002751.XSHE: ['家用轻工', '轻工制造'] ['虚拟现实'] []
3396 600048.XSHG: ['房地产开发', '房地产'] ['央企国资改革', '参股券商', '证金持股', 'MSCI概念'] []
3397 603313.XSHG: ['家用轻工', '轻工制造'] ['智能家居'] []
3398 603366.XSHG: ['家用电器', '白色家电'] ['太阳能', '家用电器', '参股民营银行', 

In [8]:
def category_dict_to_list(category_dict):
    category_list = []
    for k, v in category_dict.items():
        category_list.append(v)
    return category_list
        

ths_category = {}
for stock_info in stock_category:
    for industry in stock_info['ths_industry']:
        if industry in ths_category:
            ths_category[industry]['stocks'].append(stock_info['code'])
        else:
            ths_category[industry] = {'name': industry, 'type': 'ths_industry', 'stocks': [stock_info['code']]}
    for normal_concept in stock_info['ths_normal_concept']:
        if normal_concept in ths_category:
            ths_category[normal_concept]['stocks'].append(stock_info['code'])
        else:
            ths_category[normal_concept] = {'name': normal_concept, 'type': 'ths_normal_concept', 'stocks': [stock_info['code']]}
    for other_concept in stock_info['ths_other_concept']:
        if other_concept in ths_category:
            ths_category[other_concept]['stocks'].append(stock_info['code'])
        else:
            ths_category[other_concept] = {'name': other_concept, 'type': 'ths_other_concept', 'stocks': [stock_info['code']]}

ths_category = category_dict_to_list(ths_category)

In [9]:
col_stock_sector.delete_many({})
col_market_sector.delete_many({})

if stock_category and len(stock_category) > 0:
    col_stock_sector.insert_many(stock_category)

if ths_category and len(ths_category) > 0:
    col_market_sector.insert_many(ths_category)